In [1]:
!pip install sentence-transformers


In [2]:
from docx import Document

# Load the summary template
summary_doc = Document("Summary.docx")

# Dynamically pull every field name from the first column of each table
report_fields = []
for table in summary_doc.tables:
    for row in table.rows:
        fld = row.cells[0].text.strip()
        # skip empty rows, duplicates, or header placeholders
        if fld and fld not in report_fields and fld.lower() not in ("", "choose an item.", "click here to enter text"):
            report_fields.append(fld)

print(f"📝 Found {len(report_fields)} fields to summarize:")
for f in report_fields:
    print("  •", f)



📝 Found 74 fields to summarize:
  • Protocol Summary - Master Disclosure Document for Interventional Studies
  • The Information on this page will not be posted
  • Before using this template for authoring, refer to the supplemental instructions on Find-IT. This template is used for ALL Interventional studies that evaluate the safety, efficacy or effectiveness of a GSK product.
Master Disclosure Document (MDD) serves as the source document to disclose protocol related information across different clinical trial registers (e.g. ClinicalTrials.gov, EU Clinical Trials Information System (EU CTIS) and/or GSK/ViiV Clinical Study Register) as required by external regulations and/or GSK policy. Check in TMF for the latest version of the template before initiating a new MDD. As information from the approved MDD will be disclosed on publicly available clinical trial register(s) as required by applicable regulations and GSK policy, minimize inclusion of information that may be considered commerc

In [3]:
# Assuming you saved paragraphs list from Day 1
# If not, reload Protocol.docx again
source_doc = Document("Protocol.docx")
paragraphs = [p.text.strip() for p in source_doc.paragraphs if p.text.strip()]

print(f"Total source paragraphs: {len(paragraphs)}")


Total source paragraphs: 835


In [4]:
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained model that turns text into embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')


In [5]:
# Encode all source paragraphs into embeddings
paragraph_embeddings = model.encode(paragraphs, convert_to_tensor=True)


In [6]:
# Dictionary to store best-matched paragraph for each field
matched_paragraphs = {}

for field in report_fields:
    # Encode the field name
    field_embedding = model.encode(field, convert_to_tensor=True)

    # Calculate cosine similarity with all source paragraphs
    similarities = util.cos_sim(field_embedding, paragraph_embeddings)[0]

    # Find the index of the paragraph with the highest similarity score
    top_idx = similarities.argmax()
    best_para = paragraphs[top_idx]

    print(f"\n🟩 FIELD: {field}")
    print(f"🔍 Best Matching Paragraph:\n{best_para}\n")

    # Save the match
    matched_paragraphs[field] = best_para



🟩 FIELD: Protocol Summary - Master Disclosure Document for Interventional Studies
🔍 Best Matching Paragraph:
6.	STUDY INTERVENTION(S) AND CONCOMITANT THERAPY	46


🟩 FIELD: The Information on this page will not be posted
🔍 Best Matching Paragraph:
PAGE


🟩 FIELD: Before using this template for authoring, refer to the supplemental instructions on Find-IT. This template is used for ALL Interventional studies that evaluate the safety, efficacy or effectiveness of a GSK product.
Master Disclosure Document (MDD) serves as the source document to disclose protocol related information across different clinical trial registers (e.g. ClinicalTrials.gov, EU Clinical Trials Information System (EU CTIS) and/or GSK/ViiV Clinical Study Register) as required by external regulations and/or GSK policy. Check in TMF for the latest version of the template before initiating a new MDD. As information from the approved MDD will be disclosed on publicly available clinical trial register(s) as required by appl

In [7]:
import json

# Save the matched paragraphs from Day 2 into a file
with open("matched_paragraphs.json", "w") as f:
    json.dump(matched_paragraphs, f, indent=2)
